In this notebook we will demonstrate various CNN and RNN models for the task of intent detection on the ATIS dataset. 

In [1]:

#making the necessary imports
import os
import sys
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.initializers import Constant

from sklearn.preprocessing import LabelEncoder

import random
random.seed(0)#for reproducability of results

import pandas as pd
import numpy as np

Loading the training data

In [2]:
#utils is included in this repository'c Ch6 folder under folder name "Data"
from Data.utils import fetch_data, read_method

sents,labels,intents = fetch_data('Data/data2/atis.train.w-intent.iob')

train_sentences = [" ".join(i) for i in sents]

train_texts = train_sentences
train_labels= intents.tolist()

vals = []

for i in range(len(train_labels)):
    if "#" in train_labels[i]:
        vals.append(i)
        
for i in vals[::-1]:
    train_labels.pop(i)
    train_texts.pop(i)

print ("Number of training sentences :",len(train_texts))
print ("Number of unique intents :",len(set(train_labels)))

for i in zip(train_texts[:5], train_labels[:5]):
    print(i)

Number of training sentences : 4952
Number of unique intents : 17
('i want to fly from boston at 838 am and arrive in denver at 1110 in the morning', 'atis_flight')
('what flights are available from pittsburgh to baltimore on thursday morning', 'atis_flight')
('what is the arrival time in san francisco for the 755 am flight leaving washington', 'atis_flight_time')
('cheapest airfare from tacoma to orlando', 'atis_airfare')
('round trip fares from pittsburgh to philadelphia under 1000 dollars', 'atis_airfare')


Loading the test data

In [3]:
from Data.utils import fetch_data, read_method

sents,labels,intents = fetch_data('Data/data2/atis.test.w-intent.iob')

test_sentences = [" ".join(i) for i in sents]

test_texts = test_sentences
test_labels = intents.tolist()

new_labels = set(test_labels) - set(train_labels)

vals = []

for i in range(len(test_labels)):
    if "#" in test_labels[i]:
        vals.append(i)
    elif test_labels[i] in new_labels:
        print(test_labels[i])
        vals.append(i)
        
for i in vals[::-1]:
    test_labels.pop(i)
    test_texts.pop(i)

print ("Number of testing sentences :",len(test_texts))
print ("Number of unique intents :",len(set(test_labels)))

for i in zip(test_texts[:5], test_labels[:5]):
    print(i)

atis_day_name
atis_day_name
Number of testing sentences : 876
Number of unique intents : 15
('i would like to find a flight from charlotte to las vegas that makes a stop in st. louis', 'atis_flight')
('on april first i need a ticket from tacoma to san jose departing before 7 am', 'atis_airfare')
('on april first i need a flight going from phoenix to san diego', 'atis_flight')
('i would like a flight traveling one way from phoenix to san diego on april first', 'atis_flight')
('i would like a flight from orlando to salt lake city for april first on delta airlines', 'atis_flight')


Pre-Processing

In [4]:
MAX_SEQUENCE_LENGTH = 300
MAX_NUM_WORDS = 20000 
EMBEDDING_DIM = 100 
VALIDATION_SPLIT = 0.3

In [5]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts) #Converting text to a vector of word indexes
test_sequences = tokenizer.texts_to_sequences(test_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 897 unique tokens.


In [6]:
le = LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)
test_labels = le.transform(test_labels)

In [7]:
#Converting this to sequences to be fed into neural network. Max seq. len is 1000 as set earlier
 #initial padding of 0s, until vector is of size MAX_SEQUENCE_LENGTH
trainvalid_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
trainvalid_labels = to_categorical(train_labels)

test_labels = to_categorical(np.asarray(test_labels), num_classes= trainvalid_labels.shape[1])

# split the training data into a training set and a validation set
indices = np.arange(trainvalid_data.shape[0])
np.random.shuffle(indices)
trainvalid_data = trainvalid_data[indices]
trainvalid_labels = trainvalid_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * trainvalid_data.shape[0])
x_train = trainvalid_data[:-num_validation_samples]
y_train = trainvalid_labels[:-num_validation_samples]
x_val = trainvalid_data[-num_validation_samples:]
y_val = trainvalid_labels[-num_validation_samples:]
#This is the data we will use for CNN and RNN training
print('Splitting the train data into train and valid is done')

Splitting the train data into train and valid is done


In [8]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip

In [9]:
print('Preparing embedding matrix.')

# first, build index mapping words in the embeddings set
# to their embedding vector

# Download GloVe 6B from here: https://nlp.stanford.edu/projects/glove/
BASE_DIR = 'Data'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors in Glove embeddings.' % len(embeddings_index))
#print(embeddings_index["google"])

# prepare embedding matrix - rows are the words from word_index, columns are the embeddings of that word from glove.
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load these pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
print("Preparing of embedding matrix is done")

Preparing embedding matrix.
Found 400000 word vectors in Glove embeddings.
Preparing of embedding matrix is done


In [10]:
print('Define a 1D CNN model.')

cnnmodel = Sequential()
cnnmodel.add(embedding_layer)
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(GlobalMaxPooling1D())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dense(len(trainvalid_labels[0]), activation='softmax'))

cnnmodel.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

cnnmodel.summary()

#Train the model. Tune to validation set. 
cnnmodel.fit(x_train, y_train,
          batch_size=128,
          epochs=1, validation_data=(x_val, y_val))
#Evaluate on test set:
score, acc = cnnmodel.evaluate(test_data, test_labels)
print('Test accuracy with CNN:', acc)

Define a 1D CNN model.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          89800     
_________________________________________________________________
conv1d (Conv1D)              (None, 296, 128)          64128     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 59, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 55, 128)           82048     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 11, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 7, 128)            82048     
_________________________________________________________________
global_max_pooling1d (Global (Non

In [11]:
!rm cnnmodel_Ch6_01_cnn.h5

In [18]:
#from tensorflow import keras
from tensorflow.keras.models import load_model
cnnmodel.save("cnnmodel_Ch6_01_cnn.h5")
#del cnnmodel_reload
cnnmodel_reload=load_model("cnnmodel_Ch6_01_cnn.h5")


In [19]:
score, acc = cnnmodel.evaluate(test_data, test_labels)
print('Test accuracy with CNN:', acc)

28/28 [==============================] - 3s 88ms/step - loss: 0.9233 - acc: 0.7272
Test accuracy with CNN: 0.7271689772605896


In [20]:
score, acc = cnnmodel_reload.evaluate(test_data, test_labels)
print('Test accuracy with CNN:', acc)

28/28 [==============================] - 4s 128ms/step - loss: 0.9233 - acc: 0.7272
Test accuracy with CNN: 0.7271689772605896


In [21]:
predictions_reload = cnnmodel_reload.predict(test_data)

In [22]:
predictions_original = cnnmodel.predict(test_data)

In [23]:
predictions_reload.shape

(876, 17)

In [24]:
predictions_original.shape

(876, 17)

In [25]:
predictions_reload[0]

array([2.74056569e-03, 1.19571155e-02, 6.03919216e-02, 5.21824993e-02,
       9.10208677e-04, 1.31386355e-03, 9.41800477e-04, 6.30950148e-04,
       1.88868155e-03, 8.30409169e-01, 2.34467792e-03, 1.05756670e-02,
       2.17650924e-03, 1.60767511e-02, 1.83339987e-03, 3.25839524e-03,
       3.67942237e-04], dtype=float32)

In [26]:
predictions_original[0]

array([2.74056569e-03, 1.19571155e-02, 6.03919216e-02, 5.21824993e-02,
       9.10208677e-04, 1.31386355e-03, 9.41800477e-04, 6.30950148e-04,
       1.88868155e-03, 8.30409169e-01, 2.34467792e-03, 1.05756670e-02,
       2.17650924e-03, 1.60767511e-02, 1.83339987e-03, 3.25839524e-03,
       3.67942237e-04], dtype=float32)

In [55]:
cnnmodel.predict(test_data[0].reshape(1,300))

array([[0.01226109, 0.08262777, 0.0635693 , 0.02164723, 0.00386606,
        0.00638703, 0.00073836, 0.00411973, 0.0021619 , 0.7346508 ,
        0.00483226, 0.01642147, 0.00528206, 0.02222055, 0.00266573,
        0.01262821, 0.00392044]], dtype=float32)

In [56]:
cnnmodel_reload(test_data[0].reshape(1,300))

<tf.Tensor: shape=(1, 17), dtype=float32, numpy=
array([[0.01226109, 0.08262777, 0.0635693 , 0.02164723, 0.00386606,
        0.00638703, 0.00073836, 0.00411973, 0.0021619 , 0.7346508 ,
        0.00483226, 0.01642147, 0.00528206, 0.02222055, 0.00266573,
        0.01262821, 0.00392044]], dtype=float32)>

In [38]:
test1=()
dir(test1.index)


['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__text_signature__']

In [36]:
for i in test_data:
   test1.update(i)

TypeError: cannot convert dictionary update sequence element #0 to a sequence

In [163]:
type(test1[10])

int

In [111]:
dir(list(test1['rec1']))

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [103]:
np.array( tuple(test1['rec1']), dtype=int)

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [67]:
cnnmodel_reload.predict(np.array(test1['rec1']).reshape(1,300))

array([[0.01226109, 0.08262777, 0.0635693 , 0.02164723, 0.00386606,
        0.00638703, 0.00073836, 0.00411973, 0.0021619 , 0.7346508 ,
        0.00483226, 0.01642147, 0.00528206, 0.02222055, 0.00266573,
        0.01262821, 0.00392044]], dtype=float32)

In [43]:
print('Define a 1D CNN model.')

cnnmodel = Sequential()
cnnmodel.add(embedding_layer)
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(GlobalMaxPooling1D())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dense(len(trainvalid_labels[0]), activation='softmax'))

cnnmodel.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

cnnmodel.summary()

#Train the model. Tune to validation set. 
cnnmodel.fit(x_train, y_train,
          batch_size=128,
          epochs=24, validation_data=(x_val, y_val))
#Evaluate on test set:
score, acc = cnnmodel.evaluate(test_data, test_labels)
print('Test accuracy with CNN:', acc)

Define a 1D CNN model.
Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          89800     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 296, 128)          64128     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 59, 128)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 55, 128)           82048     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 11, 128)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 7, 128)            82048     
_________________________________________________________________
global_max_pooling1d_2 (Glob (

In [10]:
print("Defining and training a CNN model, training embedding layer on the fly instead of using pre-trained embeddings")
cnnmodel = Sequential()
cnnmodel.add(Embedding(MAX_NUM_WORDS, 128))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(GlobalMaxPooling1D())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dense(len(trainvalid_labels[0]), activation='softmax'))

cnnmodel.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

cnnmodel.summary()

#Train the model. Tune to validation set. 
cnnmodel.fit(x_train, y_train,
          batch_size=128,
          epochs=1, validation_data=(x_val, y_val))
#Evaluate on test set:
score, acc = cnnmodel.evaluate(test_data, test_labels)
print('Test accuracy with CNN:', acc)


Defining and training a CNN model, training embedding layer on the fly instead of using pre-trained embeddings
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 128)         82048     
_________

In [11]:
print("Defining and training an LSTM model, training embedding layer on the fly")

#modified from: 

rnnmodel = Sequential()
rnnmodel.add(Embedding(MAX_NUM_WORDS, 128))
rnnmodel.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel.add(Dense(len(trainvalid_labels[0]), activation='sigmoid'))
rnnmodel.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

rnnmodel.summary()

print('Training the RNN')
rnnmodel.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val))
score, acc = rnnmodel.evaluate(test_data, test_labels,
                            batch_size=32)
print('Test accuracy with RNN:', acc)


Defining and training an LSTM model, training embedding layer on the fly
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense_4 (Dense)              (None, 17)                2193      
Total params: 2,693,777
Trainable params: 2,693,777
Non-trainable params: 0
_________________________________________________________________
Training the RNN
Epoch 1/5
109/109 [==============================] - 309s 3s/step - loss: 0.2851 - accuracy: 0.6999 - val_loss: 0.1013 - val_accuracy: 0.7387
Epoch 2/5
109/109 [==============================] - 300s 3s/step - loss: 0.0976 - accuracy: 0.7493 - val_loss: 0.0983 - val_accuracy: 0.7387


In [12]:
print("Defining and training an LSTM model, using pre-trained embedding layer")

#modified from: 

rnnmodel2 = Sequential()
rnnmodel2.add(embedding_layer)
rnnmodel2.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel2.add(Dense(len(trainvalid_labels[0]), activation='sigmoid'))
rnnmodel2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

rnnmodel2.summary()

print('Training the RNN')
rnnmodel2.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val))
score, acc = rnnmodel2.evaluate(test_data, test_labels,
                            batch_size=32)
print('Test accuracy with RNN:', acc)


Defining and training an LSTM model, using pre-trained embedding layer
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          89800     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_5 (Dense)              (None, 17)                2193      
Total params: 209,241
Trainable params: 119,441
Non-trainable params: 89,800
_________________________________________________________________
Training the RNN
Epoch 1/5
109/109 [==============================] - 313s 3s/step - loss: 0.2538 - accuracy: 0.6119 - val_loss: 0.0918 - val_accuracy: 0.7387
Epoch 2/5
109/109 [==============================] - 173s 2s/step - loss: 0.0885 - accuracy: 0.7489 - val_loss: 0.0737 - val_accuracy: 0.8013
E

In [37]:
print("Defining and training an LSTM model, using pre-trained embedding layer")

#modified from: 

rnnmodel2 = Sequential()
rnnmodel2.add(embedding_layer)
rnnmodel2.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel2.add(Dense(len(trainvalid_labels[0]), activation='sigmoid'))
rnnmodel2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

rnnmodel2.summary()

print('Training the RNN')
rnnmodel2.fit(x_train, y_train,
          batch_size=32,
          epochs=8,
          validation_data=(x_val, y_val))
score, acc = rnnmodel2.evaluate(test_data, test_labels,
                            batch_size=32)
print('Test accuracy with RNN:', acc)


Defining and training an LSTM model, using pre-trained embedding layer
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          89800     
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_10 (Dense)             (None, 17)                2193      
Total params: 209,241
Trainable params: 119,441
Non-trainable params: 89,800
_________________________________________________________________
Training the RNN
Epoch 1/8
109/109 [==============================] - 113s 975ms/step - loss: 0.2415 - accuracy: 0.6326 - val_loss: 0.0833 - val_accuracy: 0.7535
Epoch 2/8
109/109 [==============================] - 103s 942ms/step - loss: 0.0827 - accuracy: 0.7632 - val_loss: 0.0628 - val_accuracy: 0.

In [38]:
print("Defining and training an LSTM model, using pre-trained embedding layer")

#modified from: 

rnnmodel2 = Sequential()
rnnmodel2.add(embedding_layer)
rnnmodel2.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel2.add(Dense(len(trainvalid_labels[0]), activation='sigmoid'))
rnnmodel2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

rnnmodel2.summary()

print('Training the RNN')
rnnmodel2.fit(x_train, y_train,
          batch_size=64,
          epochs=8,
          validation_data=(x_val, y_val))
score, acc = rnnmodel2.evaluate(test_data, test_labels,
                            batch_size=32)
print('Test accuracy with RNN:', acc)

Defining and training an LSTM model, using pre-trained embedding layer
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          89800     
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_11 (Dense)             (None, 17)                2193      
Total params: 209,241
Trainable params: 119,441
Non-trainable params: 89,800
_________________________________________________________________
Training the RNN
Epoch 1/8
55/55 [==============================] - 81s 1s/step - loss: 0.3029 - accuracy: 0.6665 - val_loss: 0.0952 - val_accuracy: 0.7535
Epoch 2/8
55/55 [==============================] - 66s 1s/step - loss: 0.1019 - accuracy: 0.7208 - val_loss: 0.0861 - val_accuracy: 0.7535
Epoch 3

In [39]:
print("Defining and training an LSTM model, using pre-trained embedding layer")

#modified from: 

rnnmodel2 = Sequential()
rnnmodel2.add(embedding_layer)
rnnmodel2.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel2.add(Dense(len(trainvalid_labels[0]), activation='sigmoid'))
rnnmodel2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

rnnmodel2.summary()

print('Training the RNN')
rnnmodel2.fit(x_train, y_train,
          batch_size=64,
          epochs=8,
          validation_data=(x_val, y_val))
score, acc = rnnmodel2.evaluate(test_data, test_labels,
                            batch_size=64)
print('Test accuracy with RNN:', acc)

Defining and training an LSTM model, using pre-trained embedding layer
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          89800     
_________________________________________________________________
lstm_8 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_12 (Dense)             (None, 17)                2193      
Total params: 209,241
Trainable params: 119,441
Non-trainable params: 89,800
_________________________________________________________________
Training the RNN
Epoch 1/8
55/55 [==============================] - 75s 1s/step - loss: 0.3115 - accuracy: 0.6264 - val_loss: 0.0951 - val_accuracy: 0.7535
Epoch 2/8
55/55 [==============================] - 98s 2s/step - loss: 0.0975 - accuracy: 0.7363 - val_loss: 0.0843 - val_accuracy: 0.7535
Epoch 

In [40]:
print("Defining and training an LSTM model, using pre-trained embedding layer")

#modified from: 

rnnmodel2 = Sequential()
rnnmodel2.add(embedding_layer)
rnnmodel2.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel2.add(Dense(len(trainvalid_labels[0]), activation='sigmoid'))
rnnmodel2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

rnnmodel2.summary()

print('Training the RNN')
rnnmodel2.fit(x_train, y_train,
          batch_size=32,
          epochs=16,
          validation_data=(x_val, y_val))
score, acc = rnnmodel2.evaluate(test_data, test_labels,
                            batch_size=32)
print('Test accuracy with RNN:', acc)

Defining and training an LSTM model, using pre-trained embedding layer
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          89800     
_________________________________________________________________
lstm_9 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_13 (Dense)             (None, 17)                2193      
Total params: 209,241
Trainable params: 119,441
Non-trainable params: 89,800
_________________________________________________________________
Training the RNN
Epoch 1/16
109/109 [==============================] - 127s 1s/step - loss: 0.2317 - accuracy: 0.6670 - val_loss: 0.0792 - val_accuracy: 0.7657
Epoch 2/16
109/109 [==============================] - 125s 1s/step - loss: 0.0864 - accuracy: 0.7606 - val_loss: 0.0585 - val_accuracy: 0.842

In [41]:
print("Defining and training an LSTM model, using pre-trained embedding layer")

#modified from: 

rnnmodel2 = Sequential()
rnnmodel2.add(embedding_layer)
rnnmodel2.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel2.add(Dense(len(trainvalid_labels[0]), activation='sigmoid'))
rnnmodel2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

rnnmodel2.summary()

print('Training the RNN')
rnnmodel2.fit(x_train, y_train,
          batch_size=32,
          epochs=32,
          validation_data=(x_val, y_val))
score, acc = rnnmodel2.evaluate(test_data, test_labels,
                            batch_size=32)
print('Test accuracy with RNN:', acc)

Defining and training an LSTM model, using pre-trained embedding layer
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          89800     
_________________________________________________________________
lstm_10 (LSTM)               (None, 128)               117248    
_________________________________________________________________
dense_14 (Dense)             (None, 17)                2193      
Total params: 209,241
Trainable params: 119,441
Non-trainable params: 89,800
_________________________________________________________________
Training the RNN
Epoch 1/32
109/109 [==============================] - 122s 1s/step - loss: 0.2362 - accuracy: 0.6666 - val_loss: 0.0885 - val_accuracy: 0.7535
Epoch 2/32
109/109 [==============================] - 104s 949ms/step - loss: 0.0898 - accuracy: 0.7409 - val_loss: 0.0662 - val_accuracy: 0.

In [42]:
print("Defining and training an LSTM model, using pre-trained embedding layer")

#modified from: 

rnnmodel2 = Sequential()
rnnmodel2.add(embedding_layer)
rnnmodel2.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel2.add(Dense(len(trainvalid_labels[0]), activation='sigmoid'))
rnnmodel2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

rnnmodel2.summary()

print('Training the RNN')
rnnmodel2.fit(x_train, y_train,
          batch_size=32,
          epochs=64,
          validation_data=(x_val, y_val))
score, acc = rnnmodel2.evaluate(test_data, test_labels,
                            batch_size=32)
print('Test accuracy with RNN:', acc)

Defining and training an LSTM model, using pre-trained embedding layer
Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          89800     
_________________________________________________________________
lstm_11 (LSTM)               (None, 128)               117248    
_________________________________________________________________
dense_15 (Dense)             (None, 17)                2193      
Total params: 209,241
Trainable params: 119,441
Non-trainable params: 89,800
_________________________________________________________________
Training the RNN
Epoch 1/64
109/109 [==============================] - 115s 989ms/step - loss: 0.2417 - accuracy: 0.6527 - val_loss: 0.0905 - val_accuracy: 0.7535
Epoch 2/64
109/109 [==============================] - 105s 963ms/step - loss: 0.0917 - accuracy: 0.7383 - val_loss: 0.0730 - val_accuracy:

In [31]:
!pip install onnxmltools onnxruntime tf2onnx keras2onnx

In [ ]:
help(onnxmltools.convert)

In [10]:
import os
os.environ['TF_KERAS']='1'

In [11]:
print(os.environ['TF_KERAS'])

1


In [22]:
!pwd

/home/hlaz001/practicalnlp/Ch6


In [124]:
cnnmodel.save('model_path2')

INFO:tensorflow:Assets written to: model_path2/assets


In [136]:
from tensorflow import keras
cnnmodel_restored = keras.models.load_model('model_path2')


In [89]:
test_labels[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [127]:
cnnmodel_restored.name

'sequential_1'

In [101]:
test_labels[0].reshape(1,17).shape

(1, 17)

In [130]:
score2, acc2 = cnnmodel.evaluate(test_data[350].reshape(1,300), test_labels[350].reshape(1,17))
print(acc2)

1/1 [==============================] - 0s 82ms/step - loss: 0.0114 - acc: 1.0000
1.0


In [86]:
len(test_data[0].reshape(1,300))

1

In [141]:
cnnmodel_restored.outputs[0].

AttributeError: 'KerasTensor' object has no attribute 'graph'

In [137]:
prediction = cnnmodel_restored.predict(test_data[0].reshape(1,300))

In [84]:
prediction.tolist()

[[0.00085111876251176,
  0.001417235704138875,
  0.006768747232854366,
  0.0035902531817555428,
  0.00019047834211960435,
  4.9524136557010934e-05,
  1.985645758395549e-05,
  8.106949098873883e-05,
  0.0002301860076840967,
  0.9799414873123169,
  0.00015161065675783902,
  0.0006940678576938808,
  8.792074368102476e-05,
  0.00517652602866292,
  4.419558172230609e-05,
  0.0006731032044626772,
  3.2594431104371324e-05]]

In [135]:
prediction2 = cnnmodel.predict(test_data[0].reshape(1,300))
prediction2.tolist()

[[0.000405784317990765,
  0.0019435544963926077,
  0.04649471864104271,
  0.0029041613452136517,
  3.46479173458647e-05,
  0.00010881847993005067,
  0.0002588485076557845,
  0.00014623059541918337,
  0.0001420097250957042,
  0.9415048956871033,
  0.0005090154008939862,
  0.0017443564720451832,
  0.00017915127682499588,
  0.0027891064528375864,
  4.667467510444112e-05,
  0.000731141131836921,
  5.677477747667581e-05]]

In [143]:
import onnxmltools
import tf2onnx
import keras2onnx

#onnx_model = onnxmltools.convert_tensorflow(cnnmodel)
#onnxmltools.utils.save_model(onnx_model, 'keras_cnn.onnx')
#onnx_model = keras2onnx.convert_keras(cnnmodel)
#keras2onnx.convert_keras(cnnmodel, cnnmodel.name)

tf executing eager_mode: True
tf.keras model eager_mode: False


AttributeError: 'KerasTensor' object has no attribute 'graph'

In [12]:
help(cnnmodel.save)

Help on method save in module tensorflow.python.keras.engine.training:

save(filepath, overwrite=True, include_optimizer=True, save_format=None, signatures=None, options=None, save_traces=True) method of tensorflow.python.keras.engine.sequential.Sequential instance
    Saves the model to Tensorflow SavedModel or a single HDF5 file.
    
    Please see `tf.keras.models.save_model` or the
    [Serialization and Saving guide](https://keras.io/guides/serialization_and_saving/)
    for details.
    
    Arguments:
        filepath: String, PathLike, path to SavedModel or H5 file to save the
            model.
        overwrite: Whether to silently overwrite any existing file at the
            target location, or provide the user with a manual prompt.
        include_optimizer: If True, save optimizer's state together.
        save_format: Either `'tf'` or `'h5'`, indicating whether to save the
            model to Tensorflow SavedModel or HDF5. Defaults to 'tf' in TF 2.X,
            and '

In [32]:
cnnmodel.save("cnnmodel_Ch6_01_cnn.h5")

In [15]:
del cnnmodel

In [17]:
from tensorflow.keras.models import load_model

In [34]:
cnnmodel_reload=load_model("cnnmodel_Ch6_01_cnn.h5")

In [33]:
del cnnmodel_reload